<div align="center"><h1>Accelerating Applications with CUDA C/C++</h1></div>

Aby przyspieszyć wykonywanie programu, należy przenieść obliczenia z CPU. Wiele przełomów powstałych dzięki akceleracji obliczeniowej, stale rosnące zapotrzebowanie na akcelerowane aplikacje, konwencje programowania ułatwiające ich pisanie oraz ciągłe ulepszanie sprzętu, który je obsługuje, napędzają tę nieuniknioną zmianę.

W centrum sukcesu przyspieszonego przetwarzania, zarówno pod względem imponującej wydajności, jak i łatwości użytkowania, znajduje się platforma obliczeniowa CUDA. CUDA zapewnia paradygmat kodowania, który rozszerza języki takie jak C, C++, Python i Fortran, aby umożliwić uruchamianie akcelerowanego, masowo zrównoleglonego kodu na najwydajniejszych na świecie procesorach równoległych: procesorach graficznych NVIDIA. CUDA drastycznie przyspiesza aplikacje przy niewielkim wysiłku, ma ekosystem wysoce zoptymalizowanych bibliotek dla DNN, BLAS, analizy wykresów, FFT i innych, a także jest dostarczany z potężnymi liniami poleceń i profilami wizualnymi.

CUDA obsługuje wiele, jeśli nie większość, najbardziej wydajnych aplikacji na świecie w: obliczeniowej dynamice płynów, dynamice molekularnej, chemii kwantowej, fizyce i HPC.

Nauka CUDA pozwoli na przyspieszenie własnych aplikacji. Przyspieszone aplikacje działają znacznie szybciej niż ich odpowiedniki wykorzystujące tylko procesory i umożliwiają obliczenia, które w innym przypadku byłyby niemożliwe do wykonania ze względu na ograniczoną wydajność aplikacji wykorzystujących tylko procesory.

---
## Cele

Po dzisiejszych zajęciach powinniście być w stanie:

- Pisać, kompilować i uruchamiać programy w języku C/C++, które zarówno wywołują funkcje na CPU oraz **uruchamiają kernele** na GPU.
- Kontrolować równoległą **hierarchię wątków** wykorzystując **konfigurację wykonania**.
- Przerabiać pętle, aby iteracje wykonywały się równolegle.
- Alokować i zwalniać pamięć dostepną zarówno dla procesorów jak i kart graficznych.
- Obsługiwać błędy generowane przez kod CUDA.
- Przyspieszać aplikacje wykorzystujące tylko CPU.

---
## Accelerated Systems

*Systemy akcelerowane*, zwane również systemami *heterogenicznymi* to te, które składają się zarówno z procesorów CPU, jak i GPU. Systemy te uruchamiają programy CPU, które z kolei uruchamiają funkcje wykorzystujące obliczenia równoległe zapewniane przez GPU. Zajęcia opierają się o taki system, który zawiera procesor graficzny NVIDIA. Informacje o tym GPU można uzyskać za pomoca polecenia wiersza poleceń `nvidia-smi` (*Systems Management Inferface*).

In [1]:
!nvidia-smi

Sat Oct 21 15:54:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

---
## GPU-accelerated Vs. CPU-only Applications

---
## Pisanie kodu aplikacji dla GPU

CUDA dostarcza rozszerzenia dla wielu popularnych języków programowania, w przypadku tych zajęć C/C++. Te rozszerzenia językowe umożliwiają programistom uruchamianie funkcji w ich kodzie źródłowym na GPU.

Poniżej znajduje się plik `.cu` (`.cu` to rozszerzenie pliku dla programów z akceleracją CUDA). Zawiera dwie funkcje, pierwszą działającą na CPU, drugą działającą na GPU. Można zauważyć różnice między funkcjami, zarówno pod względem ich definicji, jak i sposobu ich wywoływania.

```cpp
void CPUFunction()
{
  printf("This function is defined to run on the CPU.\n");
}

__global__ void GPUFunction()
{
  printf("This function is defined to run on the GPU.\n");
}

int main()
{
  CPUFunction();

  GPUFunction<<<1, 1>>>();
  cudaDeviceSynchronize();
}
```

Oto kilka ważnych linijek kodu do podkreślenia, a także kilka innych pojęć używanych w akceleracji obliczeniowej:

`__global__ void GPUFunction()`
- Słowo kluczowe `__global__` wskazuje, że następująca funkcja będzie działać na GPU i może zostać wywołana **globalnie**, co w tym kontekście oznacza albo przez CPU, albo przez GPU.
- Często kod wykonywany na procesorze jest określany jako kod **hosta** (*host*), a kod działający na GPU jest określany jako kod **urządzenia** (*device*).

`GPUFunction<<<1, 1>>>();`
- Zwykle, gdy wywołujemy funkcję uruchamianą na GPU, nazywamy tę funkcję **kernelem**, który jest **uruchamiany**.
- Podczas uruchamiania kernela musimy zapewnić **konfigurację wykonania**, która jest wykonywana przy użyciu składni `<<< ... >>>` tuż przed przekazaniem kernelowi wszelkich oczekiwanych argumentów.
- Na wysokim poziomie konfiguracja wykonania pozwala programistom określić **hierarchię wątków** do uruchomienia kernela, która definiuje liczbę grupowań wątków (zwanych **blokami**), a także liczbę **wątków** do wykonania w każdym bloku. Konfiguracja wykonywania zostanie szczegółowo omówiona później, na ten moment należy zauważyć, że kernel uruchamia się z jednym blokiem wątków `1` (pierwszy argument konfiguracji wykonania), który zawiera jeden wątek `1` (drugi argument konfiguracyjny).

`cudaDeviceSynchronize();`
- W przeciwieństwie do większości kodu C/C++, uruchamianie kernela jest **asynchroniczne**: kod procesora będzie nadal wykonywany *bez oczekiwania na zakończenie uruchamiania kernela*.
- Wywołanie `cudaDeviceSynchronize`, funkcji dostarczanej przez środowisko wykonawcze CUDA, spowoduje, że kod hosta (CPU) będzie czekał na zakończenie kodu urządzenia (GPU), a dopiero potem wznowi wykonywanie na procesorze.

---
### Ćwiczenie: Write a Hello GPU Kernel

Plik `01-hello-gpu.cu` zawiera program, który już działa. Zawiera dwie funkcje, obie z komunikatami "Hello from the CPU". Celem jest przerobienie funkcji `helloGPU` w pliku źródłowym, tak aby faktycznie działała na GPU i wyświetlała komunikat wskazujący, że działa.

In [2]:
!nvcc -arch=sm_70 -o hello-gpu 01-hello-gpu.cu -run

Hello from the CPU.
Hello also from the CPU.


Po udanej refaktoryzacji `01-hello-gpu.cu` dokonamy następujących modyfikacji, próbując skompilować i uruchomić po każdej zmianie. Otrzymamy błędy, należy poświęcić trochę czasu na ich uważne przeczytanie: ich znajomość będzie bardzo przydatna, w pisaniu własnego przyspieszonego kodu.
- Usuń słowo kluczowe `__global__` z definicji kernela. Zwróć uwagę na numer wiersza w błędzie: jak myślisz, co oznacza w błędzie „skonfigurowany”? Po zakończeniu przywróć `__global__`.
- Usuń konfigurację wykonywania: czy twoje rozumienie „skonfigurowanego” nadal ma sens? Po zakończeniu zamień konfigurację wykonania.
- Usuń wywołanie `cudaDeviceSynchronize`. Przed skompilowaniem i uruchomieniem kodu zgadnij, co się stanie, pamiętając, że kernele są uruchamiane asynchronicznie, a `cudaDeviceSynchronize` powoduje, że wykonanie hosta czeka na zakończenie wykonywania jądra przed kontynuowaniem. Po zakończeniu przywróć `cudaDeviceSynchronize`.
- Zmodyfikuj `01-hello-gpu.cu`, aby `Hello from the GPU` było wypisane **przed** Hello z CPU.
- Zmodyfikuj `01-hello-gpu.cu`, aby `Hello from the GPU` było wypisane **dwa razy**, raz **przed** `Hello from the CPU` i raz **po**.

---
### Kompilacja i uruchamianie przyspieszonego kodu CUDA

Ta sekcja zawiera szczegółowe informacje o poleceniu `nvcc`, którego użyto powyżej, aby skompilować i uruchomić program `.cu`.

Platforma CUDA zawiera [**NVIDIA CUDA Compiler**](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html) `nvcc`, który służy do kompilacji aplikacji z akceleracją CUDA, zarówno hosta jak i urządzenia. Zostaną omówione niezbędne elementy, w celu uzyskania szczegółowych informacji o `nvcc` należy zapoznać się z [dokumentacją](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html)

`nvcc` jest bardzo podobne do polecenia `gcc`. Na przykładz kopilacja jakiegoś pliku `przyklad-CUDA.cu` mogłaby wyglądać nastepująco:
`nvcc -arch=sm_70 -o out przyklad-CUDA.cu -run`
  - `nvcc` to polecenie wiersza poleceń do używania kompilatora `nvcc`.
  - `przyklad-CUDA.cu` jest przekazywany jako plik do skompilowania.
  - Flaga `o` jest używana do określenia pliku wyjściowego dla skompilowanego programu.
  - Flaga `arch` wskazuje, dla której **architektury** pliki muszą być skompilowane. Aby dopasować tą flagę do własnego GPU, należy zapoznać się z [flagą `arch`](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#options-for-steering-gpu-code-generation), [virtual architecture features](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature-list) i [GPU features](http://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature-list).
  - Podanie flagi `run` spowoduje wykonanie pomyślnie skompilowanego pliku binarnego.

---
## CUDA Thread Hierarchy

---
## Launching Parallel Kernels

Konfiguracja wykonania pozwala programistom określić szczegóły dotyczące uruchamiania kernela do równoległego działania na wielu **wątkach** GPU. Dokładniej, konfiguracja wykonania pozwala programistom określić, ile grup wątków - zwanych **blokami wątków** lub po prostu **blokami** - i ile wątków ma zawierać każdy blok wątków. Składnia tego jest następująca:

`<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS_PER_BLOCK>>>`

** Kod kernela jest wykonywany przez każdy wątek w bloku, skonfigurowanym podczas uruchamiania kernela**.

Zatem przy założeniu, że zdefiniowano kernel o nazwie `someKernel`, można powiedzieć że:
   - `someKernel<<<1, 1>>>()` jest skonfigurowany do działania w jednym bloku, który ma jeden wątek i dlatego będzie działał tylko raz.
   - `someKernel<<<1, 10>>>()` jest skonfigurowany do działania w jednym bloku, który ma 10 wątków i dlatego będzie działał 10 razy.
   - `someKernel<<<10, 1>>>()` jest skonfigurowany do działania w 10 blokach wątków, z których każdy ma jeden wątek i dlatego będzie działał 10 razy.
   - `someKernel<<<10, 10>>>()` jest skonfigurowany do działania w 10 blokach wątków, z których każdy ma 10 wątków i dlatego będzie działał 100 razy.

---
### Ćwiczenie: Launch Parallel Kernels

Plik `01-first-parallel.cu` obecnie wykonuje bardzo proste wywołanie funkcji, która wyświetla komunikat `This should be running in parallel.` Wykonaj poniższe czynności, aby najpierw dokonać refaktoryzacji w celu uruchomienia na GPU, a następnie równolegle, w jednym, a następnie w wielu blokach wątków:

- Refaktoryzacja funkcji `firstParallel`, aby uruchomić ją jako kernel CUDA na GPU. Po skompilowaniu i uruchomieniu `01-first-parallel.cu` za pomocą poniższego polecenia `nvcc` nadal powinno być widoczne wyjście funkcji.
- Refaktoryzuj kernel `firstParallel`, aby było wykonywane równolegle w 5 wątkach, wszystkie wykonywane w jednym bloku wątku. Powinieneś zobaczyć komunikat wyjściowy wydrukowany 5 razy po skompilowaniu i uruchomieniu kodu.
- Ponownie zrefaktoryzuj kernel `firstParallel`, tym razem tak, aby było wykonywane równolegle wewnątrz 5 bloków wątków, z których każdy zawiera 5 wątków. Powinieneś zobaczyć komunikat wyjściowy wydrukowany 25 razy po skompilowaniu i uruchomieniu.

In [ ]:
!nvcc -arch=sm_70 -o first-parallel 01-first-parallel.cu -run

---

## CUDA-Provided Thread Hierarchy Variables

---
## Indeksy wątków i bloków

Każdy wątek otrzymuje indeks w swoim bloku wątków, zaczynając od „0”. Dodatkowo każdy blok otrzymuje indeks, zaczynając od „0”. Podobnie jak wątki są pogrupowane w bloki, bloki są pogrupowane w **grid**, który jest najwyższą jednostką w hierarchii wątków CUDA. Podsumowując, kernele CUDA są wykonywane w gridzie z 1 lub więcej bloków, przy czym każdy blok zawiera taką samą liczbę  wątków.

Kernele CUDA mają dostęp do specjalnych zmiennych identyfikujących zarówno indeks wątku (w bloku), który wykonuje kernel, jak i indeks bloku (w gridzie), w którym znajduje się wątek. Te zmienne to odpowiednio `threadIdx.x` i `blockIdx.x`.

---
### Ćwiczenie: Używanie określonych indeksów wątków i bloków

Obecnie plik `01-thread-and-block-idx.cu` zawiera działający kernel, który wyświetla komunikat o błędzie. Otwórz plik, aby dowiedzieć się, jak zaktualizować konfigurację wykonywania, aby wyświetlić komunikat o powodzeniu.

In [7]:
!nvcc -arch=sm_70 -o thread-and-block-idx 01-thread-and-block-idx.cu -run

Success!


---
## Przyspieszanie pętli for

Pętle for w aplikacjach wykorzystujących tylko procesor są łatwe do przyspieszenia: zamiast uruchamiać każdą iterację pętli szeregowo, każda iteracja może być uruchamiana równolegle we własnym wątku. Rozważ następującą pętlę for i zauważ, że kontroluje ona, ile razy pętla zostanie wykonana, a także definiuje, co stanie się w każdej iteracji pętli:

```cpp
int N = 2<<20;
for (int i = 0; i < N; ++i)
{
  printf("%d\n", i);
}
```

Aby zrównoleglić tę pętlę, należy wykonać 2 kroki:

- Kernel musi być napisany, aby wykonać pracę **pojedynczej iteracji pętli**.
- Ponieważ kernel będzie niezależny od innych działających kerneli, konfiguracja wykonania musi być taka, aby kernel wykonał się odpowiednią liczbę razy, na przykład, ile razy wykonałaby iterację pętli.

---
### Ćwiczenie: Przyspieszenie pętli For z pojedynczym blokiem wątków

Obecnie funkcja `loop` wewnątrz `01-single-block-loop.cu` uruchamia pętlę for, która będzie wypisywać szeregowo liczby od „0” do „9”. Refaktoryzuj funkcję `loop` tak, aby była kernelem CUDA, który uruchomi się, aby wykonywać równolegle `N` iteracji. Po pomyślnej refaktoryzacji liczby od „0” do „9” nadal powinny być drukowane.

In [ ]:
!nvcc -arch=sm_70 -o single-block-loop 01-single-block-loop.cu -run

---
## Koordynowanie równoległych wątków

---
## Wykorzystywanie wymiarów bloków do większej równoległości

Istnieje ograniczenie liczby wątków, które mogą istnieć w bloku wątków: dokładnie 1024. Aby zwiększyć ilość równoległości w akcelerowanych aplikacjach, musimy być w stanie koordynować między wieloma blokami wątków.

Kernele CUDA mają dostęp do specjalnej zmiennej, która podaje liczbę wątków w bloku: `blockDim.x`. Używając tej zmiennej, w połączeniu z `blockIdx.x` i `threadIdx.x`, można uzyskać zwiększoną równoległość, organizując równoległe wykonywanie wielu bloków wielu wątków za pomocą wyrażenia `threadIdx.x + blockIdx.x * blockDim.x `. Przykład:

Konfiguracja wykonania `<<<10, 10>>>` uruchomiłaby siatkę zawierającą w sumie 100 wątków, zawartą w 10 blokach po 10 wątków. Mamy zatem nadzieję, że każdy wątek będzie mógł obliczyć pewien unikalny dla siebie indeks między „0” a „99”.

- Jeśli blok `blockIdx.x` równa się `0`, to `blockIdx.x * blockDim.x` to `0`. Dodając do `0` możliwe wartości `threadIdx.x` od `0` do `9`, możemy wygenerować indeksy od `0` do `9` w obrębie siatki 100 wątków.
- Jeśli blok `blockIdx.x` równa się `1`, to `blockIdx.x * blockDim.x` to `10`. Dodając do `10` możliwe wartości `threadIdx.x` o wartościach od `0` do `9`, możemy wygenerować indeksy od `10` do `19` w obrębie siatki 100 wątków.
- Jeśli blok `blockIdx.x` równa się `5`, to `blockIdx.x * blockDim.x` to `50`. Dodając do `50` możliwe wartości `threadIdx.x` od `0` do `9`, możemy wygenerować indeksy od `50` do `59` w obrębie siatki 100 wątków.
- Jeśli blok `blockIdx.x` równa się `9`, to `blockIdx.x * blockDim.x` to `90`. Dodając do `90` możliwe wartości `threadIdx.x` o wartościach od `0` do `9`, możemy wygenerować indeksy od `90` do `99` w obrębie siatki 100 wątków.

---
### Ćwiczenie: Przyspieszenie pętli for z wieloma blokami wątków

Obecnie funkcja `loop` wewnątrz `02-multi-block-loop.cu` uruchamia pętlę for, która seryjnie wyświetla numery od `0` do `9`. Refaktoryzuj funkcję `loop` tak, aby była kernelem CUDA, który uruchomi się, aby wykonywać równolegle `N` iteracji. Po pomyślnej refaktoryzacji liczby od `0` do `9` nadal powinny być wyświetlane. W tym ćwiczeniu jako dodatkowe ograniczenie użyj konfiguracji wykonania, która uruchamia *co najmniej 2 bloki wątków.*

In [ ]:
!nvcc -arch=sm_70 -o multi-block-loop 02-multi-block-loop.cu -run

---
## Przydzielanie pamięci, która będzie dostępna na GPU i CPU

Nowsze wersje CUDA (wersja 6 i nowsze) ułatwiły przydzielanie pamięci, która jest dostępna zarówno dla hosta procesora, jak i dowolnej liczby urządzeń GPU, i chociaż istnieje wiele [technik średniozaawansowanych i zaawansowanych](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations) do zarządzania pamięcią, która będzie obsługiwać najbardziej optymalną wydajność w akcelerowanych aplikacjach, najbardziej podstawowa technika zarządzania pamięcią CUDA, którą teraz omówimy zapewnia duże przyrosty wydajności w porównaniu z aplikacjami wykorzystującymi tylko procesor, prawie bez narzutu programisty.

Aby przydzielić i zwolnić pamięć oraz uzyskać wskaźnik, do którego można się odwoływać zarówno w kodzie hosta, jak i urządzenia, należy zastąpić wywołania `malloc` i `free` słowami `cudaMallocManaged` i `cudaFree`, jak w poniższym przykładzie:

```cpp
// CPU-only

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
a = (int *)malloc(size);

// Use `a` in CPU-only program.

free(a);
```

```cpp
// Accelerated

int N = 2<<20;
size_t size = N * sizeof(int);

int *a;
// Note the address of `a` is passed as first argument.
cudaMallocManaged(&a, size);

// Use `a` on the CPU and/or on any GPU in the accelerated system.

cudaFree(a);
```

---
### Ćwiczenie: dostęp do tablicy zarówno z poziomu hosta jak i urządzenia

Program `01-double-elements.cu` alokuje tablicę, inicjuje ją wartościami całkowitymi na hoście, próbuje podwoić każdą z  wartości równolegle na GPU, a następnie potwierdza, czy operacje podwajania powiodły się, na hoście. Obecnie program nie będzie działał: próbuje współdziałać zarówno na hoście, jak i na urządzeniu z tablicą ze wskaźnikiem `a`, ale przydzielił tylko tablicę (za pomocą `malloc`), aby była dostępna na hoście. Wskazówki:

- `a` powinno być dostępne zarówno dla kodu hosta, jak i urządzenia.
- Pamięć w `a` powinna być poprawnie zwolniona.

In [ ]:
!nvcc -arch=sm_70 -o double-elements 01-double-elements.cu -run

## Grid Size Work Amount Mismatch

---
## Obsługa niezgodności konfiguracji bloku z liczbą potrzebnych wątków

Może się zdarzyć, że nie można zapisać konfiguracji wykonania, która utworzy dokładną liczbę wątków potrzebnych do zrównoleglenia pętli.

Typowy przykład dotyczy chęci wyboru optymalnych rozmiarów bloków. Na przykład, ze względu na cechy sprzętowe procesora graficznego, bloki zawierające wiele wątków będących wielokrotnością 32 są często pożądane w celu zwiększenia wydajności. Zakładając, że chcielibyśmy uruchomić bloki, z których każdy zawierał 256 wątków (wielokrotność 32) i musieliśmy uruchomić 1000 równoległych zadań (trywialnie mała liczba dla ułatwienia wyjaśnienia), to nie ma takiej liczby bloków, która dawałaby dokładną sumę 1000 wątków w siatce, ponieważ nie ma wartości całkowitej, która pomnożona przez 32 da równe 1000.

Ten scenariusz można łatwo rozwiązać w następujący sposób:

- Napisz konfigurację wykonania, która tworzy **więcej** wątków niż jest to konieczne do wykonania przydzielonej pracy.
- Przekaż wartość jako argument do kernela (`N`), która reprezentuje całkowity rozmiar zestawu danych do przetworzenia lub całkowitą liczbę wątków potrzebnych do ukończenia pracy.
- Po obliczeniu indeksu wątku w gridzie (za pomocą `tid+bid*bdim`), sprawdź, czy ten indeks nie przekracza `N`, a jeśli tak nie jest, wykonaj odpowiednią pracę kernela.

Oto przykład sposobu pisania konfiguracji wykonania, gdy znane są zarówno `N`, jak i liczba wątków w bloku i nie można zagwarantować dokładnego dopasowania między liczbą wątków w gridzie a `N`. Zapewnia, że zawsze jest co najmniej tyle wątków, ile potrzeba dla `N` i maksymalnie tylko 1 dodatkowy blok dodatkowych wątków:

```cpp
// Assume `N` is known
int N = 100000;

// Assume we have a desire to set `threads_per_block` exactly to `256`
size_t threads_per_block = 256;

// Ensure there are at least `N` threads in the grid, but only 1 block's worth extra
size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;

some_kernel<<<number_of_blocks, threads_per_block>>>(N);
```

Ponieważ powyższa konfiguracja wykonania skutkuje większą liczbą wątków w gridzie niż `N`, należy zachować ostrożność wewnątrz definicji `some_kernel`, aby `some_kernel` nie próbowało uzyskać dostępu do elementów danych spoza zakresu podczas wykonywania przez jeden z "dodatkowych" wątków:

```cpp
__global__ some_kernel(int N)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;

  if (idx < N) // Check to make sure `idx` maps to some value within `N`
  {
    // Only do work if it does
  }
}
```

---
### Ćwiczenie: Przyspieszenie pętli For z niedopasowaną konfiguracją wykonywania

Program w `02-mismatched-config-loop.cu` alokuje pamięć, używając `cudaMallocManaged` dla 1000-elementowej tablicy liczb całkowitych, a następnie próbuje zainicjować wszystkie wartości tablicy równolegle przy użyciu kernela CUDA. Ten program zakłada, że znane są zarówno `N` jak i liczba `threads_per_block`. Twoim zadaniem jest:

- Przypisanie wartości do `number_of_blocks`, która upewni się, że jest co najmniej tyle wątków, ile jest elementów w `a` do pracy.
- Zaktualizowanie kernela `initializeElementsTo`, aby upewnić się, że nie próbuje wykonywać operacji na elementach danych, które są poza zakresem.

In [ ]:
!nvcc -arch=sm_70 -o mismatched-config-loop 02-mismatched-config-loop.cu -run

---
## Grid-Stride Loops

---
## Zbiór danych większy niż grid

Albo z wyboru, często w celu stworzenia najbardziej wydajnej konfiguracji wykonania, albo z konieczności, liczba wątków w gridzie może być mniejsza niż rozmiar zbioru danych. Rozważ tablicę z 1000 elementów i grid z 250 wątkami (używając tutaj trywialnych rozmiarów dla ułatwienia wyjaśnienia). Tutaj każdy wątek w gridzie będzie musiał zostać użyty 4 razy. Jedną z popularnych metod, aby to zrobić, jest użycie **pętli grid-stride** w kernelu.

W pętli grid-stride każdy wątek obliczy swój unikalny indeks w gridzie za pomocą `tid+bid*bdim`, wykona operację na elemencie o tym indeksie w tablicy, a następnie doda do swojego indeksu liczbę wątków w gridzie i powtarza operacje, aż znajdzie się poza zasięgiem tablicy. Na przykład dla tablicy 500 elementów i grid 250 wątków, wątek z indeksem 20 w gridzie:

- Wykonaj operację na elemencie 20 z 500 elementów tablicy
- Zwiększ jego indeks o 250, rozmiar grid, co daje 270
- Wykonaj operację na elemencie 270 z 500 elementów tablicy
- Zwiększ swój indeks o 250, rozmiar grid, co daje 520
- Ponieważ 520 jest teraz poza zasięgiem tablicy, wątek przestanie działać

CUDA dostarcza specjalną zmienną podającą liczbę bloków w siatce, `gridDim.x`. Obliczenie całkowitej liczby wątków w siatce to po prostu liczba bloków w siatce pomnożona przez liczbę wątków w każdym bloku, `gridDim.x * blockDim.x`. Mając to na uwadze, oto szczegółowy przykład pętli grid-stride w kernelu:

```cpp
__global__ void kernel(int *a, int N)
{
  int indexWithinTheGrid = threadIdx.x + blockIdx.x * blockDim.x;
  int gridStride = gridDim.x * blockDim.x;

  for (int i = indexWithinTheGrid; i < N; i += gridStride)
  {
    // do work on a[i];
  }
}
```

---
### Ćwiczenie: wykorzystywanie pętli grid-stride

Refaktoryzuj `03-grid-stride-double.cu`, aby użyć pętli grid-stride w kernelu `doubleElements` w kolejności że grid, który jest mniejszy niż `N`, może ponownie wykorzystać wątki, aby pokryć każdy element tablicy. Program wypisze czy każdy element w tablicy został podwojony, obecnie program dokładnie wypisuje `FALSE`.

In [ ]:
!nvcc -arch=sm_70 -o grid-stride-double 03-grid-stride-double.cu -run

---
## Obsługa błędów

Jak w każdej aplikacji, obsługa błędów w przyspieszonym kodzie CUDA jest niezbędna. Wiele, jeśli nie większość funkcji CUDA (patrz na przykład [funkcje zarządzania pamięcią](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY)) zwraca wartość typu `cudaError_t`, którego można użyć do sprawdzenia, czy wystąpił błąd podczas wywoływania funkcji. Oto przykład, w którym obsługa błędów jest wykonywana dla wywołania `cudaMallocManaged`:

```cpp
cudaError_t err;
err = cudaMallocManaged(&a, N)                    // Assume the existence of `a` and `N`.

if (err != cudaSuccess)                           // `cudaSuccess` is provided by CUDA.
{
  printf("Error: %s\n", cudaGetErrorString(err)); // `cudaGetErrorString` is provided by CUDA.
}
```

Uruchamiane kerneli, które są zdefiniowane jako zwracające `void`, nie zwracają wartości typu `cudaError_t`. Aby sprawdzić błędy występujące podczas uruchamiania kernela, na przykład jeśli konfiguracja uruchamiania jest błędna, CUDA udostępnia funkcję `cudaGetLastError`, która zwraca wartość typu `cudaError_t`.

```cpp
/*
 * This launch should cause an error, but the kernel itself
 * cannot return it.
 */

someKernel<<<1, -1>>>();  // -1 is not a valid number of threads.

cudaError_t err;
err = cudaGetLastError(); // `cudaGetLastError` will return the error from above.
if (err != cudaSuccess)
{
  printf("Error: %s\n", cudaGetErrorString(err));
}
```

Aby wychwycić błędy, które występują asynchronicznie, na przykład podczas wykonywania asynchronicznego kernela, konieczne jest sprawdzenie stanu zwróconego przez kolejne synchronizujące wywołanie API środowiska wykonawczego CUDA, takie jak `cudaDeviceSynchronize`, które zwróci błąd, jeśli jedno z uruchomionych wcześniej kerneli powinno zawieść.


---
### Ćwiczenie: dodawanie obsługi błędów

Obecnie `01-add-error-handling.cu` kompiluje, uruchamia i wyświetla, że elementy tablicy nie zostały pomyślnie podwojone. Program nie wskazuje jednak, że są w nim jakieś błędy. Refaktoryzuj aplikację do obsługi błędów CUDA, dzięki czemu możesz dowiedzieć się, co jest nie tak z programem i skutecznie go debugować. Będziesz musiał zbadać zarówno błędy synchroniczne potencjalnie tworzone podczas wywoływania funkcji CUDA, jak i błędy asynchroniczne potencjalnie tworzone podczas wykonywania jądra CUDA.

In [ ]:
!nvcc -arch=sm_70 -o add-error-handling 01-add-error-handling.cu -run

---
### CUDA Error Handling Function

Pomocne może być utworzenie makra, które otacza wywołania funkcji CUDA w celu sprawdzenia błędów. Oto przykład, możesz go użyć w pozostałych ćwiczeniach:

```cpp
#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

int main()
{

/*
 * The macro can be wrapped around any function returning
 * a value of type `cudaError_t`.
 */

  checkCuda( cudaDeviceSynchronize() )
}
```

---
## Podsumowanie

Po zajęciach powinniście być w stanie:

- Pisać, kompilować i uruchamiać programy w języku C/C++, które zarówno wywołują funkcje procesora, jak i **uruchamiają** **kernele** GPU.
- Kontrolować równoległą **hierarchię wątków** za pomocą **konfiguracji wykonywania**.
- Refaktoryzować pętle szeregowe, aby wykonywać ich iteracje równolegle na GPU.
- Przydzielać i zwalniać pamięć dostępną zarówno dla procesorów, jak i kart graficznych.
- Obsługiwać błędy generowane przez kod CUDA.

Pozwoli to na wykonanie ostatniego zadania:

- Przyspiesz aplikacje wykorzystujące tylko procesor.

---
### Ćwiczenie końcowe: Przyspiesz aplikację dodawania wektorów

Poniższe zadanie wymaga wykorzystania wszystkiego, czego nauczyliście się do tej pory. Polega ono na przyspieszeniu programu dodawania wektorów tylko przez procesor, który choć nie jest najbardziej wyrafinowanym programem, daje możliwość skupienia się na tym, czego nauczyliście się o przyspieszaniu aplikacji przez GPU za pomocą CUDA.

`01-vector-add.cu` zawiera działającą aplikację dodawania wektorów tylko dla procesora CPU. Przyspiesz jego funkcję `addVectorsInto`, aby działała jako kernel CUDA na GPU i wykonywała swoją pracę równolegle. **Rozwiązanie należy przesłać na michal.zimon@wat.edu.pl**. Wskazówki:

- Rozszerz definicję `addVectorsInto` tak, aby był to kernel CUDA.
- Wybierz i wykorzystaj działającą konfigurację wykonania, aby `addVectorsInto` uruchamiał się jako kernel CUDA.
- Aktualizuj alokacje pamięci i zwalnianie pamięci, aby odzwierciedlić, że 3 wektory `a`, `b` i `result` muszą być dostępne przez kod hosta i urządzenia.
- Refaktoryzacja `addVectorsInto`: zostanie uruchomiony wewnątrz pojedynczego wątku i wystarczy wykonać tylko jeden wątek na wektorach wejściowych. Upewnij się, że wątek nigdy nie będzie próbował uzyskać dostępu do elementów spoza zakresu wektorów wejściowych i zwróć uwagę, czy wątek musi pracować na więcej niż jednym elemencie wektorów wejściowych.
- Dodaj obsługę błędów w lokalizacjach, w których kod CUDA może nie działać.



In [ ]:
!nvcc -arch=sm_70 -o vector-add 01-vector-add.cu -run